In [1]:
import os

import cupy as cp
import pandas as pd
import torch

import cudf
import dask_cudf

cp.random.seed(12)

#### Portions of this were borrowed and adapted from the
#### cuDF cheatsheet, existing cuDF documentation,
#### and 10 Minutes to Pandas.

In [2]:
import torch
import cudf
import timeit
from cuml.dask.neighbors import NearestNeighbors
from cuml.common.device_selection import using_device_type, set_global_device_type, get_global_device_type

In [3]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)

/home/jabl91/miniconda3/envs/pointcept_py_3_9/lib/python3.9/site-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(


In [4]:
# Generate a "coordinate" dataframe
n_points = 32000
df = cudf.DataFrame()

point_cloud = torch.cuda.FloatTensor(n_points, 3).uniform_()

df['x'] = point_cloud[:, 0]
df['y'] = point_cloud[:, 1]
df['z'] = point_cloud[:, 2]

In [5]:
ddf = dask_cudf.from_cudf(df, npartitions=1)
ddf.head()

ddf = ddf.persist()

In [6]:
n_neighbors = 16

# Create a cuML NearestNeighbors model
nn = NearestNeighbors(n_neighbors=n_neighbors, client=client)

# Compute the Dask DataFrame
# ddf_computed = ddf.compute()

start_time = timeit.default_timer()

# Fit the model with the input data
nn.fit(ddf)

# Get the nearest neighbors
distances, indices = nn.kneighbors(ddf)


end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"auto Execution time: {execution_time} seconds")

[1710116980.565040] [DESKTOP-FAOOUT6:15532:0]          parser.c:1993 UCX  WARN  unused environment variable: UCX_CUDA_COPY_MAX_REG_RATIO
[1710116980.565040] [DESKTOP-FAOOUT6:15532:0]          parser.c:1993 UCX  WARN  (set UCX_WARN_UNUSED_ENV_VARS=n to suppress this warning)
auto Execution time: 1.0705611229968781 seconds


In [7]:
# Create a cuML NearestNeighbors model
nn2 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='rbc', client=client)
# Compute the Dask DataFrame
#ddf_computed = ddf.compute()

start_time = timeit.default_timer()



# Fit the model with the input data
nn2.fit(ddf)

# Get the nearest neighbors
distances, indices = nn2.kneighbors(ddf)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"rbc Execution time: {execution_time} seconds")

rbc Execution time: 0.2507939189999888 seconds


In [8]:
# Create a cuML NearestNeighbors model
nn3 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ivfflat', client=client)
# Compute the Dask DataFrame
#ddf_computed = ddf.compute()

start_time = timeit.default_timer()



# Fit the model with the input data
nn3.fit(ddf)

# Get the nearest neighbors
distances, indices = nn3.kneighbors(ddf)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"ivfflat Execution time: {execution_time} seconds")

ivfflat Execution time: 0.18994943799771136 seconds


In [9]:
import torch
import pointops

n_points_tensor = torch.cuda.FloatTensor([n_points])

start_time = timeit.default_timer()

reference_index, _ = pointops.knn_query(n_neighbors, point_cloud, n_points_tensor)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"C++ PointopsV1 KNN Execution time: {execution_time} seconds")

C++ PointopsV1 KNN Execution time: 0.0023637989979761187 seconds


In [10]:
import torch
from pointops2 import pointops2 as newerPointOps2

start_time = timeit.default_timer()

reference_index, _ = newerPointOps2.knnquery(n_neighbors, point_cloud, None, n_points_tensor, n_points_tensor)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"C++ PointopsV2 KNN Execution time: {execution_time} seconds")

C++ PointopsV2 KNN Execution time: 0.0020435499973245896 seconds


In [11]:
# # Create a cuML NearestNeighbors model
# nn4 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ivfpq')
# # Compute the Dask DataFrame
# ddf_computed = ddf.compute()

# start_time = timeit.default_timer()



# # Fit the model with the input data
# nn4.fit(ddf_computed)

# # Get the nearest neighbors
# distances, indices = nn4.kneighbors(ddf_computed)

# end_time = timeit.default_timer()
# execution_time = end_time - start_time
# print(f"ivfflat Execution time: {execution_time} seconds")

In [12]:
client.close()